<a href="https://colab.research.google.com/github/SherbyRobotics/pyro/blob/colab/examples/notebooks/phase_plane_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Phase Plane

This page is an interactive tutorial on phase plane analysis for dynamical systems.

## Tutorial content

1.   What is a phase plane?
2.   Exemple: a floating mass
3.   Exemple: a pendulum
4.   Exemple: your custom dynamic system!
5.   What about closed-loop systems?
6.   Exemple: a floating mass with a linear controller








## Importing Pyro

We will use a custom toolbox called pyro.


In [ ]:
!git clone https://github.com/SherbyRobotics/pyro
import sys
sys.path.append('/content/pyro')
import pyro


Here we import other basic python tools:

*   Numpy: the python library for linear algebra, on top of which pyro is built. 
*   Display: that is needed to show animation in the colab environment. If pyro is used locally then this is not needed.
*   Inspect: that we will use here only for printing source code in for this tutorial



In [ ]:
import numpy as np
from IPython import display
import inspect

# What is a phase plane? (or phase portrait)



In [ ]:
display.IFrame("https://www.youtube.com/embed/QGPVxZrZq44",800,600)

# A Simple Floating Mass

Here we load a class that already includes the dynamic equation:

$m \ddot{q} + b \dot{q} = f $

i.e. a mass with some linear damping $b$ and an external force input $f$.

In [ ]:
from pyro.dynamic import massspringdamper

m = 1 # Mass parameter
b = 0 # Damping parameter

sys = massspringdamper.FloatingSingleMass( m , b )

Where the state vector is defined as 

$x = \begin{bmatrix} q \\ \dot{q} \end{bmatrix}$

In [ ]:
print(sys.state_label)
print(sys.state_units)

The toolbox as built-in functions to plot phase planes for dynamic system objects:

In [ ]:
sys.plot_phase_plane()

All those blue arrow represents how the system states would evolve in time. If we start a trajectory at a given point the system states would "follow" the arrows.

Lets simulate a specific trajectory:

In [ ]:
sys.x0 = np.array([0,5])             # Initial conditions
sys.compute_trajectory( tf = 1 )     # Simulate for 1 sec
sys.plot_phase_plane_trajectory()    # Plot the trajectory superposed on the phase plane

The black dot is the initial state, the red x is the final states and the blue line is the path. Here we see that for a single mass with no damping, if we start at a position $q=0$ and velocity $\dot{q}=5$ then the system would continue with constant velocity (y-axis on the phase plane) but the position would increase continuously (x-axis on the phase plane).

## Now with damping

Here lets see how adding damping change the behavior:



In [ ]:
m = 1   # Mass parameter
b = 0.5 # Damping parameter
sys = massspringdamper.FloatingSingleMass( m , b )
sys.x0 = np.array([0,5])             # Initial conditions
sys.compute_trajectory( tf = 1 )     # Simulate for 1 sec
sys.plot_phase_plane_trajectory()    # Plot the trajectory superposed on the phase plane

Here we see that now velocity naturally decrease (arrows point toward the horizontal $\dot{q}=0$ line.

## Now with a constant force (zero damping againg)


In [ ]:
m = 1   # Mass parameter
b = 0   # Damping parameter
sys = massspringdamper.FloatingSingleMass( m , b )
sys.ubar[0]  =  -5.0                 # A constant default force always applied on the mass
sys.x0 = np.array([0,5])             # Initial conditions
sys.compute_trajectory( tf = 3 )     # Simulate for 1 sec
sys.plot_phase_plane_trajectory()    # Plot the trajectory superposed on the phase plane

Here we see that the behavior is caracterised by arcs (constant acceleration trajectories). To make more sense of the simulated trajectory lets animate it:

In [ ]:
ani  = sys.generate_simulation_html_video()
html = display.HTML( ani )
display.display(html)

Here we see the for this trajectory the constant force first slow down the mass until a rest point put continue accelerating it in the negative direction.

 Now I recommend for you to go back and play with the parameters and initial conditions to check how it affect the behavior.

# The Single Pendulum

The floating mass is caracterized by a simple linear behavior. Phase-plane are particulary interesting for analysing non-linear system. Let's analyse the dynamic of a pendulum.

Here load a pendulum class that implement the dynamic:

$ml^2 \ddot{q} + b \dot{q}  + m g l \sin q = \tau$

and then use the toolbox built-in function to plot the phase plane:

In [ ]:
from pyro.dynamic import pendulum   # Here we load pyro library of pedulum system
sys = pendulum.SinglePendulum()     # Here we create an instance of the Single Pendulum
sys.plot_phase_plane()

Now lets analyse this nice vector field with specific trajectories.

First starting with a high initial velocity, the pendulum will loop arround, slowing down at the top and accelerating arround the bottom position.

In [ ]:
sys.x0 = np.array([-6.28,5])        # Initial conditions
sys.compute_trajectory( tf = 3 )
sys.plot_phase_plane_trajectory()

In [ ]:
# This generate an animation of the trajectory
ani  = sys.generate_simulation_html_video()
html = display.HTML( ani )
display.display(html)

Now if we start with a small initial velocity, the pendulum will only oscillate arround the bottom position.

In [ ]:
sys.x0 = np.array([0,1])
sys.compute_trajectory( tf = 5 )
sys.plot_phase_plane_trajectory()

In [ ]:
# This generate an animation of the trajectory
ani  = sys.generate_simulation_html_video()
html = display.HTML( ani )
display.display(html)

## Now with damping

Now lets try again with damping!

In [ ]:
sys.d1 = 1. # damping parameter

sys.x0 = np.array([0,5])
sys.compute_trajectory( tf = 10 )
sys.plot_phase_plane_trajectory()

We see that the momentum quickly dissipate and the pendulum will converge eventually toward the bottom stable position.

In [ ]:
# This generate an animation of the trajectory
ani  = sys.generate_simulation_html_video()
html = display.HTML( ani )
display.display(html)

# A Phase Plane for A Custom Dynamic System

Now we played with dynamic systems that were already coded in the toolbox. Here lets use the toolbox to plot a phase plane for your own custom dynamic system.

First a "Dynamic System Class" must be defined. Bellow, we define the minimum necessary for using the phase plane tool: labels and the dynamic function:

$\dot{x} = f(x,u,t)$

where $x$ is the state vector, $u$ is an input vector and $t$ is time.

In [ ]:
from pyro.dynamic import system


class CustomSys( system.ContinuousDynamicSystem ):
    
    ############################
    def __init__(self):
        
        # initialize standard 2 state (n=2) dynamic system
        system.ContinuousDynamicSystem.__init__( self, n = 2 )
        
        #######################################
        # Your system label bellow:
        self.name = 'My custom system'
        self.state_label = [ 'State 1' , 'State 2' ]
        self.state_units = [ '[]', '[]']
        #######################################
    
    #############################
    def f( self , x , u , t ):
        """ 
        Continuous time foward dynamics evaluation dx = f(x,u,t)
        
        INPUTS
        x  : state vector             n x 1
        u  : control inputs vector    m x 1
        t  : time                     1 x 1
        
        OUPUTS
        dx : state derivative vector  n x 1
        
        """
        
        dx = np.zeros(self.n) # State derivative vector
        
        #######################################
        # Your Dynamic Equation bellow:
        dx[0] = -x[0] # This is a place holder
        dx[1] = -x[1] # This is a place holder
        #######################################
        
        return dx

Now you just have to create an instance and call the method!



In [ ]:
sys = CustomSys()

sys.plot_phase_plane()

You can use the previous code as a template for analysing a system you have the dynamic equations.

# Adding a Feedback controller

Now lets discuss what happen to the phase plane when we add feedback.

First, starting from an open-loop system with equations:

$\dot{x} = f(x,u)$

If we close the loop with a control law:

$u = c(x)$

We can find a new dynamic that depends only on the states:

$\dot{x} = f(x,u) = f(x, c(x))  = f_{cl}(x)$

We will call $f_{cl}$ the "Closed-loop" dynamic. We can apply the same phase-plane analysis on this new function to analyze the new behavior when a controller is active.

## Exemple

Lets use the floating mass with no damping again as the exemple:

In [ ]:
m = 1
b = 0
sys = massspringdamper.FloatingSingleMass( m , b )

This set the output equation to $y = \begin{bmatrix} q \\ \dot{q} \end{bmatrix}$ (full state feedback), because the defaut output equation was only the position $q$.

In [ ]:
sys.p = 2
sys.C = np.diag([1,1])
sys.cost_function = None     # This avoid a bug that I need to fix!

Now we will try a controller that implement a linear feedback law of the form:

$u = K ( r - y )$

where $K$ is a $1 \times 2 $ matrix of gains, $u$ is the force input that the controller will apply, $r$ is the reference (a desired state vector) and $y$ is the actual measured state vector.

In [ ]:
from pyro.control import linear

ctl = linear.ProportionalController( m = 1 , p = 2)

Next we can set feedback gains in the matrix, and a default constant reference signal:

In [ ]:
ctl.K[0,0] = 1.0
ctl.K[0,1] = 0.0
print( 'Controller gain matrix K =', ctl.K )
ctl.rbar[0] = 0
print( 'r =', ctl.rbar )

Then we create a new dynamic system object that represent the closed-loop behavior of the original system with the controller. This function basically compute $\dot{x} = f(x,u) = f(x, c(x))  = f_{cl}(x)$

In [ ]:
cl_sys = ctl + sys

This new "closed-loop" object instance can now be used like a regular dynamic system object, all the tools demonstrated in the previous section of the tutorial are available.

This illustrate the closed-loop behavior with a phase plane we can call the same method to illustrate now $f_{cl}$

In [ ]:
cl_sys.plot_phase_plane()

This behavior is caracteristic of undamped oscillations, hence this control law would need to be modified for a adequate closed-loop behavior.

# How does the control law gains affect the phase plane?

First lets try only a gain on the velocity:

In [ ]:
ctl.K[0,0]  = 0.0
ctl.K[0,1]  = 1.0
ctl.rbar[0] = 0
cl_sys.plot_phase_plane()

This would lead to bring the mass to rest but not on a desired position.

Only a gain on the position:

In [ ]:
ctl.K[0,0]  = 1.0
ctl.K[0,1]  = 0.0
ctl.rbar[0] = 0
cl_sys.plot_phase_plane()

This would lead to oscillations arround the desired position.

Lets try gains on both the position and the velocity:

In [ ]:
ctl.K[0,0]  = 1.0
ctl.K[0,1]  = 0.7
ctl.rbar[0] = 0
cl_sys.plot_phase_plane()

This is more adequate, all trajectories converge on the desired state at [0,0].

Note that changing the reference $r$ in the control law will offset the vector field:

In [ ]:
ctl.K[0,0]  = 1.0
ctl.K[0,1]  = 0.7
ctl.rbar[0] = 2.0           # New reference position
cl_sys.plot_phase_plane()

Let simulate a specific trajectory for this closed-loop system.

In [ ]:
cl_sys.x0 = np.array([0,5])
cl_sys.compute_trajectory( tf = 10 )
cl_sys.plot_phase_plane_trajectory()

and show an animation:

In [ ]:
# This would work locally in a python console
#cl_sys.animate_simulation()

# This is the way for showing an animation on colab (we need to generate html)
ani  = cl_sys.generate_simulation_html_video()
html = display.HTML( ani )
display.display(html)

# The End

I would recommend playing with parameters in all the previous interactive code blocks to make sure you understand how each parameter affect the behavior and the phase plane plots.